<a href="https://colab.research.google.com/github/VikasSharma707/LGMVIP-Data-Science/blob/main/NextWordPredictionModel_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LGM-VIP Data Science Internship Programme 2021**
## **ADVANCED LEVEL TASK**
# **Name: Next Word Prediction**

**libraries**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [ ]:
path = 'nextword.txt'

**Preprocessing the Data**

In [ ]:
file = open("nextword.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:200]

"Project Gutenberg's The Adventures of Sherlock Holmes, by Arthur Conan Doyle This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever. You may copy it, give i"

In [ ]:
print(len(data))

573660


In [ ]:
tokenizer = Tokenizer(num_words=len(data), oov_token='<OOV>')
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[143, 4681, 2, 987, 6, 126, 34, 47, 557, 2165, 2166, 28, 988, 15, 23]

In [ ]:
print(len(sequence_data))

108958


In [ ]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8625


In [ ]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  108955


array([[ 143, 4681,    2,  987],
       [4681,    2,  987,    6],
       [   2,  987,    6,  126],
       [ 987,    6,  126,   34],
       [   6,  126,   34,   47],
       [ 126,   34,   47,  557],
       [  34,   47,  557, 2165],
       [  47,  557, 2165, 2166],
       [ 557, 2165, 2166,   28],
       [2165, 2166,   28,  988]])

In [ ]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [ ]:
print("Data: ", X[:10])
print("Response: ", y[:10])

Data:  [[ 143 4681    2]
 [4681    2  987]
 [   2  987    6]
 [ 987    6  126]
 [   6  126   34]
 [ 126   34   47]
 [  34   47  557]
 [  47  557 2165]
 [ 557 2165 2166]
 [2165 2166   28]]
Response:  [ 987    6  126   34   47  557 2165 2166   28  988]


In [ ]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

**MODEL**

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             86250     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8625)              8633625   
                                                                 
Total params: 21,768,875
Trainable params: 21,768,875
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=20, batch_size=64, callbacks=[checkpoint])

Epoch 1/20
1702/1703 [============================>.] - ETA: 0s - loss: 6.3896
Epoch 00001: loss improved from inf to 6.38954, saving model to next_words.h5
1703/1703 [==============================] - 65s 34ms/step - loss: 6.3895
Epoch 2/20
1703/1703 [==============================] - ETA: 0s - loss: 5.8027
Epoch 00002: loss improved from 6.38954 to 5.80273, saving model to next_words.h5
1703/1703 [==============================] - 59s 35ms/step - loss: 5.8027
Epoch 3/20
1702/1703 [============================>.] - ETA: 0s - loss: 5.4754
Epoch 00003: loss improved from 5.80273 to 5.47527, saving model to next_words.h5
1703/1703 [==============================] - 59s 35ms/step - loss: 5.4753
Epoch 4/20
1703/1703 [==============================] - ETA: 0s - loss: 5.2078
Epoch 00004: loss improved from 5.47527 to 5.20780, saving model to next_words.h5
1703/1703 [==============================] - 59s 35ms/step - loss: 5.2078
Epoch 5/20
1703/1703 [==============================] - ETA: 0s 

**Prediction**

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

['Adventures']
of
['Arthur', 'Conan']
i
